In [1]:
data Address =
    Address {
        street :: String,
        number :: Int,
        city :: String,
        zipcode :: String
    }
    deriving Show

In [2]:
address1 = Address "Hinter der Grieb" 10 "Regensburg" "93047"

In [3]:
address1

Address {street = "Hinter der Grieb", number = 10, city = "Regensburg", zipcode = "93047"}

In [4]:
address1 { number = number address1 + 1 }

Address {street = "Hinter der Grieb", number = 11, city = "Regensburg", zipcode = "93047"}

In [5]:
data Status = Employee Int | Intern deriving Show

data Person =
    Person {
        name :: String,
        status :: Status,
        address :: Address
    }
    deriving Show

In [28]:
person1 = Person "Herr Gravenreuth" (Employee 100) address1

In [29]:
person1

Person {name = "Herr Gravenreuth", status = Employee 100, address = Address {street = "Hinter der Grieb", number = 10, city = "Regensburg", zipcode = "93047"}}

In [30]:
person1 {
        address = (address person1) {
            number = number (address person1) + 1
        }
    }

Person {name = "Herr Gravenreuth", status = Employee 100, address = Address {street = "Hinter der Grieb", number = 11, city = "Regensburg", zipcode = "93047"}}

In [31]:
data Lens s a =
    Lens {
        lget :: s -> a,
        lset :: a -> s -> s
    }

lmodify :: Lens s a -> (a -> a) -> (s -> s)
lmodify (Lens get set) f s = set (f (get s)) s

In [32]:
personAddress :: Lens Person Address
personAddress = Lens {
        lget = address,
        lset = \a p -> p { address = a }
    }
    
personName :: Lens Person String
personName = Lens {
        lget = name,
        lset = \n p -> p { name = n }
    }
    
personStatus :: Lens Person Status
personStatus = Lens {
        lget = status,
        lset = \n p -> p { status = n }
    }
    
    
addressNumber :: Lens Address Int
addressNumber = Lens {
        lget = number,
        lset = \n a -> a { number = n }
    }

In [33]:
import Data.Char
lmodify personName (map toUpper) person1

Person {name = "HERR GRAVENREUTH", status = Employee 100, address = Address {street = "Hinter der Grieb", number = 10, city = "Regensburg", zipcode = "93047"}}

In [34]:
lcompose :: Lens s a -> Lens a b -> Lens s b
lcompose (Lens get1 set1) (Lens get2 set2) = Lens {
        lget = get2 . get1,
        lset = \b s ->
                    let
                        a = get1 s
                        a' = set2 b a
                        s' = set1 a' s
                    in s'
    }

In [35]:
personAddressNumber = personAddress `lcompose` addressNumber

In [36]:
lget personAddressNumber person1

10

In [37]:
lmodify personAddressNumber (+1) person1

Person {name = "Herr Gravenreuth", status = Employee 100, address = Address {street = "Hinter der Grieb", number = 11, city = "Regensburg", zipcode = "93047"}}

In [38]:
data Prism s a = Prism {
        pget :: s -> Maybe a,
        pback :: a -> s
    }

In [39]:
statusEmployee :: Prism Status Int
statusEmployee = Prism {
        pget = \s -> do Employee salary <- Just s; return salary,
        pback = Employee
    }

In [40]:
pget statusEmployee (Employee 100)
pget statusEmployee Intern
pback statusEmployee 10

Just 100

Nothing

Employee 10

In [41]:
import Control.Monad ((>=>))
pcompose :: Prism s a -> Prism a b -> Prism s b
pcompose (Prism get1 back1) (Prism get2 back2) = Prism {
        pget = get1 >=> get2,
        pback = back1 . back2
    }

In [42]:
data Optional s a = Optional {
        oget :: s -> Maybe a,
        oset :: a -> s -> s
    }

In [43]:
prismToOptional :: Prism s a -> Optional s a
prismToOptional (Prism pget pback) = Optional {
        oget = pget,
        oset = const . pback
    }

In [44]:
lensToOptional :: Lens s a -> Optional s a
lensToOptional (Lens lget lset) = Optional {
        oget = Just . lget,
        oset = lset
    }

In [45]:
ocompose :: Optional s a -> Optional a b -> Optional s b
ocompose (Optional get1 set1) (Optional get2 set2) = Optional {
        oget = get1 >=> get2,
        oset = \b s -> case get1 s of
                            Nothing -> s
                            Just a -> set1 (set2 b a) s
    }

In [46]:
:t lensToOptional personStatus

lensToOptional personStatus :: Optional Person Status

In [47]:
:t prismToOptional statusEmployee

prismToOptional statusEmployee :: Optional Status Int

In [49]:
personEmployee = lensToOptional personStatus `ocompose` prismToOptional statusEmployee

In [52]:
oget personEmployee person1
oset personEmployee 200 person1

Just 100

Person {name = "Herr Gravenreuth", status = Employee 200, address = Address {street = "Hinter der Grieb", number = 10, city = "Regensburg", zipcode = "93047"}}

In [53]:
person2 = lset personStatus Intern person1

In [54]:
person2

Person {name = "Herr Gravenreuth", status = Intern, address = Address {street = "Hinter der Grieb", number = 10, city = "Regensburg", zipcode = "93047"}}

In [56]:
oget personEmployee person2
oset personEmployee 200 person2

Nothing

Person {name = "Herr Gravenreuth", status = Employee 200, address = Address {street = "Hinter der Grieb", number = 10, city = "Regensburg", zipcode = "93047"}}

In [58]:
fstLens :: Lens (a, b) a
fstLens = Lens { lget = fst, lset = \a' (a, b) -> (a', b) }
sndLens :: Lens (a, b) b
sndLens = Lens { lget = snd, lset = \b' (a, b) -> (a, b') }

In [60]:
nilPrism :: Prism [a] ()
nilPrism = Prism { pget = \xs -> case xs of [] -> Just (); _ -> Nothing, pback = const [] }
consPrism :: Prism [a] (a, [a])
consPrism = Prism { pget = \xs -> case xs of [] -> Nothing; (x:xs) -> Just (x, xs), pback = \(x, xs) -> x:xs }

In [66]:
headOptional :: Optional [a] a
headOptional = prismToOptional consPrism `ocompose` lensToOptional fstLens
tailOptional :: Optional [a] [a]
tailOptional = prismToOptional consPrism `ocompose` lensToOptional sndLens

In [67]:
oget headOptional [1, 2]
oget tailOptional [1, 2]

Just 1

Just [2]

In [68]:
:set -XNPlusKPatterns

In [70]:
indexOptional :: Int -> Optional [a] a
indexOptional 0 = headOptional
indexOptional (n+1) = tailOptional `ocompose` indexOptional n

In [73]:
oget (indexOptional 2) [1, 2]

Nothing

In [76]:
oset (indexOptional 2) 5 [1, 2]

[1,2]

In [80]:
atLens :: Eq a => a -> Lens (a -> b) b
atLens a = Lens {
        lget = \f -> f a,
        lset = \b f a' -> if a == a' then b else f a'
    }

In [81]:
f :: Int -> String
f = const ""

In [87]:
lget (atLens 2) (lset (atLens 3) "a" f)

""

In [88]:
data Iso s a = Iso {
        iget :: s -> a,
        iback :: a -> s
    }

In [89]:
newtype Zipcode = Zipcode String

In [93]:
prismLawGetAfterBack p a = pget p (pback p a) == Just a
prismLawBackAfterGet p s a =
    case pget p s of
        Just a -> pback p a == s
        Nothing -> True

In [95]:
optionalLawSetAfterSet = undefined
optionalLawGetAfterSet p a s = (const a <$> oget p s) == oget p (oset p a s)